In [1]:
#!/usr/bin/env python3
import pandas as pd
from pathlib import Path

def check_gdelt_folder(base_dir: str):
    base = Path(base_dir)
    raw_dir = base / "raw"
    flt_dir = base / "filtered"
    man_dir = base / "_manifest"

    print(f"[INFO] Checking folder: {base.resolve()}")

    # 1. Đếm file
    n_raw = len(list(raw_dir.glob("*.parquet"))) if raw_dir.exists() else 0
    n_flt = len(list(flt_dir.glob("*.parquet"))) if flt_dir.exists() else 0
    n_man = len(list(man_dir.glob("*.json"))) if man_dir.exists() else 0
    print(f"  Raw files: {n_raw}")
    print(f"  Filtered files: {n_flt}")
    print(f"  Manifest files: {n_man}")

    # 2. Load thử một số file filtered
    counts = []
    if flt_dir.exists():
        for p in sorted(flt_dir.glob("*.parquet")):
            try:
                df = pd.read_parquet(p)
                counts.append((p.name, len(df)))
            except Exception as e:
                counts.append((p.name, f"ERR {e}"))

    if counts:
        total_rows = sum(c for _, c in counts if isinstance(c, int))
        empty_files = [f for f, c in counts if c == 0]
        print(f"  Total rows in filtered: {total_rows}")
        print(f"  Empty filtered files: {len(empty_files)}")
        if empty_files:
            print("   → " + ", ".join(empty_files[:10]) + (" ..." if len(empty_files) > 10 else ""))
    else:
        print("  (no filtered files to check)")


# ví dụ chạy
check_gdelt_folder("../../work/data/gdelt_0921_hardfix_split")


[INFO] Checking folder: /home/kylh/phd/tw_fin_rl/work/data/gdelt_0921_hardfix_split
  Raw files: 7400
  Filtered files: 7402
  Manifest files: 7400
  Total rows in filtered: 892044
  Empty filtered files: 421
   → gdelt_filtered_BNB_2020-08-23_0900_2100_4375a3ad25d1f05a.parquet, gdelt_filtered_BNB_2020-09-20_0900_2100_3bbd5cd00e89efe9.parquet, gdelt_filtered_BNB_2020-09-27_0900_2100_044bee2002c53b37.parquet, gdelt_filtered_BNB_2020-10-11_0900_2100_d74505221e4dbb8e.parquet, gdelt_filtered_BNB_2020-10-13_0900_2100_4f0fc760b100c72d.parquet, gdelt_filtered_BNB_2020-10-20_0900_2100_66b9f0255f45f4be.parquet, gdelt_filtered_BNB_2020-10-23_0900_2100_5a19e2f9ea0cd744.parquet, gdelt_filtered_BNB_2020-10-25_0900_2100_b13af1c0794c7305.parquet, gdelt_filtered_BNB_2020-10-28_0900_2100_a0796ad45a2e1f43.parquet, gdelt_filtered_BNB_2020-10-29_0900_2100_8490277540f72e54.parquet ...


In [6]:
#!/usr/bin/env python3
"""
Quick check for MERGED parquet files:
- Tự động tìm các file có 'merged' trong tên hoặc nằm trong thư mục 'merged/'
- Đếm số dòng mà không load toàn bộ (dùng pyarrow ParquetFile)
- In thống kê: số file, tổng số dòng, số file rỗng, liệt kê vài file rỗng

Usage:
  python check_merged_parquet.py /path/to/base_dir
  # tuỳ chọn: chỉ định glob riêng
  python check_merged_parquet.py /data --glob "**/merged/*.parquet"
"""
import sys
from pathlib import Path

try:
    import pyarrow.parquet as pq
except ImportError:
    print("[ERROR] Please install pyarrow: pip install pyarrow", file=sys.stderr)
    sys.exit(1)

def count_rows_parquet(fp: Path) -> int:
    try:
        pf = pq.ParquetFile(fp)
        md = pf.metadata
        return sum(md.row_group(i).num_rows for i in range(md.num_row_groups))
    except Exception as e:
        print(f"[WARN] Can't read {fp}: {e}")
        return -1  # unreadable

def looks_merged(p: Path) -> bool:
    name_has = 'merged' in p.name.lower()
    folder_has = any(part.lower() == 'merged' for part in p.parts)
    return name_has or folder_has

def find_merged_files(base: Path, custom_glob: str | None) -> list[Path]:
    if custom_glob:
        return sorted(base.glob(custom_glob))
    # Mặc định: hai pattern phổ biến
    found = set()
    for pat in ("**/*merged*.parquet", "**/merged/*.parquet"):
        for fp in base.glob(pat):
            found.add(fp)
    # Fallback: toàn bộ parquet rồi lọc bằng looks_merged
    if not found:
        found = {p for p in base.glob("**/*.parquet") if looks_merged(p)}
    return sorted(found)

def main():
    if len(sys.argv) < 2:
        print("Usage: python check_merged_parquet.py <base_dir> [--glob \"**/merged/*.parquet\"]")
        sys.exit(1)

    base = Path(sys.argv[1]).expanduser().resolve()
    custom_glob = None
    if len(sys.argv) >= 4 and sys.argv[2] == "--glob":
        custom_glob = sys.argv[3]

    merged_files = find_merged_files(base, custom_glob)
    print(f"[INFO] Checking MERGED under: {base}")
    print(f"  Merged files: {len(merged_files)}")

    total_rows = 0
    empty_names = []
    unreadable = 0

    for fp in merged_files:
        rows = count_rows_parquet(fp)
        if rows < 0:
            unreadable += 1
            continue
        total_rows += rows
        if rows == 0 and len(empty_names) < 30:
            empty_names.append(fp.name)

    print(f"  Total rows in merged: {total_rows}")
    print(f"  Empty merged files: {len(empty_names)}")
    if empty_names:
        print("   → " + ", ".join(empty_names) + (" ..." if len(empty_names) == 30 else ""))
    if unreadable:
        print(f"  [WARN] Unreadable merged files: {unreadable}")

import sys

# giả lập lệnh:
# python check_merged_parquet.py /home/kylh/phd/tw_fin_rl/work/data/gdelt_full_hardfix_split
sys.argv = [
    "check_merged_parquet.py",
    "/home/kylh/phd/tw_fin_rl/work/data/gdelt_0921_hardfix_split"
]

# gọi hàm main trong notebook
main()



[INFO] Checking MERGED under: /home/kylh/phd/tw_fin_rl/work/data/gdelt_0921_hardfix_split
  Merged files: 2
  Total rows in merged: 717152
  Empty merged files: 0


In [8]:
import pandas as pd
from pathlib import Path

def analyze_merged(base_dir: str, glob_pat: str = "**/*merged*.parquet"):
    base = Path(base_dir).expanduser().resolve()
    files = sorted(base.glob(glob_pat))

    print(f"[INFO] Loading MERGED files from: {base}")
    # print list files 
    print("FILES:: ", files)

    dfs = []
    for fp in files:
        print(f"[INFO] Loading {fp} ...")
        try:
            df = pd.read_parquet(fp)
            dfs.append(df)
        except Exception as e:
            print(f"  [WARN] Can't read {fp}: {e}")
    if not dfs:
        return None
    
    data = pd.concat(dfs, ignore_index=True)
    
    # Xác định token từ cột 'symbol' (nếu có)
    if "symbol" in data.columns:
        tokens = data["symbol"].unique().tolist()
    else:
        # fallback: đoán từ text
        tokens = ["BTC","ETH","BNB","SOL"]

    results = {}
    for token in tokens:
        # chọn subset theo cột 'symbol'
        if "symbol" in data.columns:
            sub = data[data["symbol"] == token]
        else:
            mask = data["text"].str.contains(token, case=False, na=False)
            sub = data[mask]
        
        if sub.empty:
            continue

        results[token] = {
            "n_articles": len(sub),
            "time_range": (sub["date"].min(), sub["date"].max()) if "date" in sub.columns else None,
            "sources": sub["source"].nunique() if "source" in sub.columns else None,
            "langs": sub["lang"].value_counts().head(5).to_dict() if "lang" in sub.columns else None,
            "sample_titles": sub["title"].dropna().head(3).tolist() if "title" in sub.columns else None
        }
    return results, data

# Chạy:
stats, df_all = analyze_merged("/home/kylh/phd/tw_fin_rl/work/data/gdelt_0921_hardfix_split")

import pprint
pprint.pprint(stats)


[INFO] Loading MERGED files from: /home/kylh/phd/tw_fin_rl/work/data/gdelt_0921_hardfix_split
FILES::  [PosixPath('/home/kylh/phd/tw_fin_rl/work/data/gdelt_0921_hardfix_split/filtered/gdelt_filtered_merged_2020-08-11_2025-09-03.parquet'), PosixPath('/home/kylh/phd/tw_fin_rl/work/data/gdelt_0921_hardfix_split/filtered/gdelt_raw_merged_2020-08-11_2025-09-03.parquet')]
[INFO] Loading /home/kylh/phd/tw_fin_rl/work/data/gdelt_0921_hardfix_split/filtered/gdelt_filtered_merged_2020-08-11_2025-09-03.parquet ...
[INFO] Loading /home/kylh/phd/tw_fin_rl/work/data/gdelt_0921_hardfix_split/filtered/gdelt_raw_merged_2020-08-11_2025-09-03.parquet ...
{'BNB': {'langs': None,
         'n_articles': 12524,
         'sample_titles': ['DEX Aggregator 1inch Raises $2 . 8M From Binance '
                           'Labs , Galaxy Digital and More',
                           'Binance Denies Report It Was Blocked From '
                           'Installing Its CEO on Board of Failing Bank',
                

In [9]:
import pandas as pd

# Đường dẫn file
path = "/home/kylh/phd/tw_fin_rl/work/data/gdelt_0921_hardfix_split/filtered/gdelt_raw_merged_2020-08-11_2025-09-03.parquet"

# Đọc file parquet
df = pd.read_parquet(path)

# Thông tin cơ bản
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

# Hiển thị sample
print(df.sample(5, random_state=42))


Shape: (593758, 10)
Columns: ['title', 'url', 'domain', 'language', 'seendate', 'socialimage', 'sourcecountry', 'isduplicate', 'day', 'symbol_req']
                                                    title  \
382771  Safe ( SAFE ) Trading Down 46 . 1 % Over Last ...   
167244  A Las Vegas Adult Club Accepts Bitcoin for Lap...   
529984         SafeMoon ( SAFEMOON ) Price Tops $0 . 0000   
18007   Bitcoin on Exchanges Drop to Lows Not Seen Sin...   
327195  Biggest Week For Bitcoin And Crypto In 2023 Ahead   

                                                      url  \
382771  https://www.modernreaders.com/news/2023/10/01/...   
167244  https://www.thestreet.com/investing/a-las-vega...   
529984  https://theenterpriseleader.com/2024/10/27/saf...   
18007                     https://upstract.com/p/cuv23qtg   
327195              https://biztoc.com/x/d3cf7d41a21dde82   

                         domain language                  seendate  \
382771        modernreaders.com  English 2023-10

In [10]:
import pandas as pd
from tabulate import tabulate

# Đường dẫn file
path = "/home/kylh/phd/tw_fin_rl/work/data/gdelt_0921_hardfix_split/filtered/gdelt_raw_merged_2020-08-11_2025-09-03.parquet"

# Đọc file parquet
df = pd.read_parquet(path)

# Lấy sample
sample = df.sample(5, random_state=42)

# In bảng đẹp
print(tabulate(sample, headers="keys", tablefmt="pretty", showindex=True))


+--------+---------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+-------------------------+----------+---------------------------+---------------------------------------------------------------------------+---------------+-------------+------------+------------+
|        |                                            title                                            |                                                url                                                 |         domain          | language |         seendate          |                                socialimage                                | sourcecountry | isduplicate |    day     | symbol_req |
+--------+---------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------

In [11]:
import pandas as pd
from tabulate import tabulate

# Load dữ liệu
path = "/home/kylh/phd/tw_fin_rl/work/data/gdelt_0921_hardfix_split/filtered/gdelt_raw_merged_2020-08-11_2025-09-03.parquet"
df = pd.read_parquet(path)

# Các token cần lọc
tokens = ["BTC", "BNB", "ETH", "SOL"]

for sym in tokens:
    subset = df[df["symbol_req"] == sym]
    print(f"\n=== {sym} | total={len(subset)} ===")
    if len(subset) == 0:
        print("⚠️ Không có dữ liệu.")
        continue
    
    # Lấy 3 dòng đầu và 3 dòng cuối
    sample = pd.concat([subset.head(3), subset.tail(3)])
    
    # In bảng pretty
    print(tabulate(sample, headers="keys", tablefmt="pretty", showindex=True))



=== BTC | total=318078 ===
+--------+-------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+----------+---------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+-------------+------------+------------+
|        |                                                             title                                                             |                                                                                               url                                                                                       